In [3]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/zaq9/case/master/case20190109/data20190109.csv',index_col=0)
df

,PRICE
C20000,475
C20125,370
C20250,270
C20375,180
C20500,105
C20625,60
C20750,32
C20875,15
C21000,7
P20000,27


In [4]:
! pip install pulp

In [0]:
import pulp as pu


class Payoff():
    def __init__(self, name):
        self.type = name[0]
        self.strike = int(name[1:])
    def __call__(self, u):
        v = u - self.strike
        if(self.type =='C'): 
            return v if v>0 else 0
        else:
            return -v if v<0 else 0

In [7]:
v_sq = 20125

df["buy"] = df.index.map(lambda x: pu.LpVariable(x, 0, 10, pu.LpInteger))
df["sell"] = df.index.map(lambda x: pu.LpVariable(x+'_s', 0, 10, pu.LpInteger))
df["payoff"] = df.index.map(lambda x: Payoff(x))


def pay(s, objs): return pu.lpSum(pu.lpDot(objs, [p(s) for p in df.payoff]))
def v(s):   return pay(s, df['buy']) - pay(s, df['sell'])

cost = pu.lpSum(pu.lpDot(df['buy'], df['PRICE'])) -pu.lpSum(pu.lpDot(df['sell'], df['PRICE']))
m = pu.LpProblem('Max', sense=pu.LpMaximize)
m += v(v_sq ) -cost
m += cost <= 100
m += pu.lpSum(df['buy']) <= 10  # 合計売り枚数

for sq in range(18000,23000,125): m += v(sq) >=0 
    
m.solve()
df['ans'] = df.buy.map(lambda x: x.value()) - df.sell.map(lambda x: x.value())
print(f"最大収益 ={m.objective.value():.0f}(@SQ={v_sq}):コスト={sum(df.ans * df.PRICE):.0f}")

for x in df.query( ' ans != 0 ').itertuples():
    print(f"{x.Index}[{x.ans:.0f}]@{x.PRICE}  >> {x.payoff(v_sq)}")

最大収益 =792(@SQ=20125):コスト=83
C20000[6]@475  >> 125
C20125[-9]@370  >> 0
C20375[3]@180  >> 0
P20125[-1]@43  >> 0
P20250[1]@66  >> 125
